In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config('spark.driver.memory','3g').getOrCreate()
spark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,241 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,773 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40

In [6]:
# Citirea fișierelor de tip CSV / JSON / Parquet într-un Data Frame

# data_df = spark.read.format('csv').option('header', 'true').schema(data_schema).load('/path/to/folder/or/file')
# data_df = spark.read.format('json').schema(data_schema).load('/path/to/folder/or/file')

path = '/content/drive/MyDrive/Colab Notebooks/Data/practice/parquet'
data_df = spark.read.format('parquet').load(path)

In [7]:
# data_df.printSchema()               # Afișare tipurilor de date
# data_list = data_df.collect()       # Colectarea datelor într-o listă de Python
data_df.show(5)                          # Afișare datelor la consolă
# data_pdf = data_df.toPandas() # Colectarea datelor într-o tabelă de Pandas

+---------+------+--------------------+-------+-------+--------------------+
|     nume|varsta|            ocupatie|vechime|inactiv|               extra|
+---------+------+--------------------+-------+-------+--------------------+
|    Daria|    33|   Inginer  mecanic |      7|   null|                [EV]|
|  Delia  |    20|        Medic primar|      0|   true|                null|
|    Diana|    45|   Asistent  social |     23|   null|[XBOX, 3D Printer...|
|    Doina|    37|Specialist  marke...|     11|   null|          [WII, PS5]|
|   Elena |    25|            Contabil|      1|   null|                null|
+---------+------+--------------------+-------+-------+--------------------+
only showing top 5 rows



In [ ]:
# Scrierea datelor în fișiere de tip CSV / JSON / Parquet

# data_df.write.format('csv').option('header', 'true').save('/path/to/save/folder')
# data_df.write.format('json').save('/path/to/save/folder')
# data_df.write.format('parquet').save('/path/to/save/folder')


In [5]:
# Expresii de calcul  ## o clasă specială de Python cu un nume intuitiv, numită Column. reține doar „formula” care trebuie efectuată

from pyspark.sql import functions as f

# expr = f.lit(130)         # o valoare constantă
# expr = f.col('varsta')  # valoarea unei o coloane

In [ ]:
# FUNCTII - pentru operații pe caractere sau liste și alte structuri complexe
## https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

# f.contains()
# f.trim()
# f.date_trunc('DAY',f.current_date())
# f.size()
# f.array_remove(f.col('nn'),'string')



In [8]:
# Adăugarea sau Înlocuirea unui coloane

data_df.withColumn(
    'text',
    f.concat(
        f.lit('in varsta de '),
        f.col('varsta'),
        f.lit(' ani')
        )
    ).show(5)


+---------+------+--------------------+-------+-------+--------------------+-------------------+
|     nume|varsta|            ocupatie|vechime|inactiv|               extra|               text|
+---------+------+--------------------+-------+-------+--------------------+-------------------+
|    Daria|    33|   Inginer  mecanic |      7|   null|                [EV]|in varsta de 33 ani|
|  Delia  |    20|        Medic primar|      0|   true|                null|in varsta de 20 ani|
|    Diana|    45|   Asistent  social |     23|   null|[XBOX, 3D Printer...|in varsta de 45 ani|
|    Doina|    37|Specialist  marke...|     11|   null|          [WII, PS5]|in varsta de 37 ani|
|   Elena |    25|            Contabil|      1|   null|                null|in varsta de 25 ani|
+---------+------+--------------------+-------+-------+--------------------+-------------------+
only showing top 5 rows



In [ ]:
# convertim rezultatul expresiei într-un alt tip de date.

# expr = (f.col('varsta') + f.lit(1)).cast('string')

In [ ]:
# seta numele obiectului Column explicit.

# expr = f.expr('concat("in varsta de ", varsta, " ani")').alias('text_varsta')

In [ ]:
## Condiții în lanț de IF-ELSE

# print(
#     f.when(
#         f.col('varsta') < 25,
#         f.lit('I'))
#     .when(
#         f.col('varsta') < 32,
#         f.lit('II'))
#     .otherwise(f.lit('III')  )
#     )

In [ ]:
# transformări de date în funcția de select.

# data_df.select(
#     'nume',
#     'extra',
#     f.col('varsta') - f.col('vechime'),
#     f.concat(
#         f.lit('in varsta de '),
#         f.col('varsta'),
#         f.lit(' ani')).alias('text')
# )

# enriched_data_df = (
#     data_df
#     .withColumn(
#         'inactiv',
#         f.coalesce(
#             f.col('inactiv'),
#             f.lit(False)))
#     .withColumn(
#         'varsta_contractare',
#         f .col('varsta') - f.col('vechime'))
# )

In [ ]:
# Filtrare - Păstrarea datelor pe baza unei condiții

# data_df.filter(f.col('varsta') < 0)
# data_df.where(f.col('varsta') < 0)


In [ ]:
# Sortare

# data_df.sort(
#     'nume',
#     f.desc(
#         f.col('varsta')
#         )
#     )

In [28]:
# Limitarea numărului de rânduri

data_df.limit(2)
data_df.sort(f.desc('vechime')).limit(2)

enriched_data_df = (
    data_df
    .withColumn('inactiv', f.coalesce(f.col('inactiv'), f.lit(False)))
    .withColumn('varsta_contractare', f .col('varsta') - f.col('vechime'))
    .filter('inactiv is False')
    .sort(f.desc('vechime'))
    .limit(10)
).show(5, truncate = False)

+------+------+---------------+-------+-------+----------------------------+------------------+
|nume  |varsta|ocupatie       |vechime|inactiv|extra                       |varsta_contractare|
+------+------+---------------+-------+-------+----------------------------+------------------+
|Viorel|50    | Farmacist     |32     |false  |[PS5, 3D Printer, PC]       |18                |
|Bogdan|50    |Farmacist      |32     |false  |[PC, 3D Printer, AC]        |18                |
|Carmen|50    |Farmacist      |32     |false  |[3D Printer, AC, 3D Printer]|18                |
|Cosmin|49    |Farmacist      |31     |false  |[AC, XBOX, PV]              |18                |
|Lucian|49    |Designer grafic|30     |false  |[PV, XBOX, AC]              |19                |
+------+------+---------------+-------+-------+----------------------------+------------------+
only showing top 5 rows



In [29]:
# Nivelare a datelor
# descompune un singur rând de date în mai multe rânduri
# În setul de date nou, se va afla câte un rând de date pentru fiecare element a listei.

expr = f.explode('extra')
data_df.withColumn('consumator',expr).show(5,truncate=False)
# .drop(f.col('extra'))

+-----+------+----------------------+-------+-------+----------------------+----------+
|nume |varsta|ocupatie              |vechime|inactiv|extra                 |consumator|
+-----+------+----------------------+-------+-------+----------------------+----------+
|Daria|33    |Inginer  mecanic      |7      |null   |[EV]                  |EV        |
|Diana|45    |Asistent  social      |23     |null   |[XBOX, 3D Printer, PV]|XBOX      |
|Diana|45    |Asistent  social      |23     |null   |[XBOX, 3D Printer, PV]|3D Printer|
|Diana|45    |Asistent  social      |23     |null   |[XBOX, 3D Printer, PV]|PV        |
|Doina|37    |Specialist  marketing |11     |null   |[WII, PS5]            |WII       |
+-----+------+----------------------+-------+-------+----------------------+----------+
only showing top 5 rows



In [68]:
df_customers = spark.read.format('parquet').load('/content/drive/MyDrive/Colab Notebooks/Data/Course_III/customers_cleaned/practice/parquet')
df_tariff           = spark.read.format('parquet').load('/content/drive/MyDrive/Colab Notebooks/Data/Course_III/tariff/practice/parquet')
df_zone_source  = spark.read.format('parquet').load('/content/drive/MyDrive/Colab Notebooks/Data/Course_III/zone_source/parquet')
df_zone_type        = spark.read.format('parquet').load('/content/drive/MyDrive/Colab Notebooks/Data/Course_III/zone_type/parquet')

In [34]:
df_customers.show(5,truncate=False)

+--------+------+------------------+-------+-------+----+---------------------------+
|nume    |varsta|ocupatie          |vechime|inactiv|zona|extra                      |
+--------+------+------------------+-------+-------+----+---------------------------+
|Gabriel |44    |Specialist vânzări|21     |null   |E   |[5G Router, Sauna, PV]     |
|George  |46    |Agent imobiliar   |25     |false  |G   |[5G Router, 5G Router, WII]|
|Gheorghe|34    |Specialist HR     |8      |null   |D   |[PC, Sauna]                |
|Grigore |44    |Specialist vânzări|21     |null   |E   |[WII, Piscina, Piscina]    |
|Horia   |42    |Arhitect          |18     |null   |E   |[Piscina, AC]              |
+--------+------+------------------+-------+-------+----+---------------------------+
only showing top 5 rows



In [17]:
df_customers.printSchema()

root
 |-- nume: string (nullable = true)
 |-- varsta: integer (nullable = true)
 |-- ocupatie: string (nullable = true)
 |-- vechime: integer (nullable = true)
 |-- inactiv: boolean (nullable = true)
 |-- zona: string (nullable = true)
 |-- extra: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [35]:

df_tariff.show(5,truncate=False)


+---------+--------+
|nume     |tarif   |
+---------+--------+
|Andrei   |dinamic |
|Alexandru|dinamic |
|Adrian   |zonal   |
|Alin     |standard|
|Anton    |zonal   |
+---------+--------+
only showing top 5 rows



In [36]:
df_zone_source.show(5,truncate=False)


+----+--------+---+
|zona|sursa   |ani|
+----+--------+---+
|D   |nucleara|20 |
|B   |nucleara|16 |
|C   |eoliana |3  |
|D   |fosila  |45 |
|C   |fosila  |45 |
+----+--------+---+
only showing top 5 rows



In [38]:
df_zone_type.show(5,truncate=False)

+----+--------+
|zona|mediu   |
+----+--------+
|D   |Suburban|
|B   |Suburban|
|A   |Urban   |
|C   |Urban   |
|E   |Rural   |
+----+--------+



In [22]:
# Group by
# returnează un obiect special de tip Grouped Data
# oferă diverse metode pentru a executa operații de agregare

df_customers.groupby('zona')


In [23]:
df_customers.groupby('zona').max().show()   # max - metoda de agregate a obiectului GroupedData


+----+-----------+------------+
|zona|max(varsta)|max(vechime)|
+----+-----------+------------+
|   E|         51|          23|
|   B|         34|          11|
|   D|         44|          24|
|   C|         45|          21|
|   A|         40|          27|
|   G|         50|          32|
+----+-----------+------------+



In [54]:
df_customers.groupby('zona').max('varsta').show()  # max - metoda de agregate a obiectului GroupedData
df_customers.groupby('zona').max('vechime').show()

+----+-----------+
|zona|max(varsta)|
+----+-----------+
|   E|         51|
|   B|         34|
|   D|         44|
|   C|         45|
|   A|         40|
|   G|         50|
+----+-----------+

+----+------------+
|zona|max(vechime)|
+----+------------+
|   E|          23|
|   B|          11|
|   D|          24|
|   C|          21|
|   A|          27|
|   G|          32|
+----+------------+



In [24]:
 # group by mai multe coloane
 # sum - metoda de agregate a obiectului GroupedData
df_customers.groupby('zona', 'inactiv').sum().show(5)

+----+-------+-----------+------------+
|zona|inactiv|sum(varsta)|sum(vechime)|
+----+-------+-----------+------------+
|   G|  false|        337|         200|
|   B|  false|         27|           2|
|   E|   null|       1193|         532|
|   B|   null|        474|          42|
|   D|   null|        628|         197|
+----+-------+-----------+------------+
only showing top 5 rows



In [59]:
df_customers\
    .groupby('zona', 'inactiv')\   # group by mai multe coloane
    .sum('varsta')\
    .show(5)

+----+-------+-----------+
|zona|inactiv|sum(varsta)|
+----+-------+-----------+
|   G|  false|        337|
|   B|  false|         27|
|   E|   null|       1193|
|   B|   null|        474|
|   D|   null|        628|
+----+-------+-----------+
only showing top 5 rows



In [62]:
# Pentru agregari diverse si complexe in aceeasi comanda folosim
# agg()

df_customers \
      .groupby('zona', 'inactiv') \
      .agg(
              f.sum('varsta').alias('sum de varsta'),
              f.mean('varsta').cast('decimal(5,2)').alias('mean de varsta'),
              f.max('varsta').alias('max de varsta')
                        ) \
      .show(5)  # group by mai multe coloane

+----+-------+-------------+--------------+-------------+
|zona|inactiv|sum de varsta|mean de varsta|max de varsta|
+----+-------+-------------+--------------+-------------+
|   G|  false|          337|         48.14|           50|
|   B|  false|           27|         27.00|           27|
|   E|   null|         1193|         42.61|           45|
|   B|   null|          474|         26.33|           34|
|   D|   null|          628|         36.94|           44|
+----+-------+-------------+--------------+-------------+
only showing top 5 rows



In [58]:
# Functii de agregare

from pyspark.sql import functions as f


In [63]:
expr1 = f.first('vechime', ignorenulls=True).alias('fst_vechime')
df_customers.groupby('zona', 'inactiv').agg(expr1).show(5)

+----+-------+-----------+
|zona|inactiv|fst_vechime|
+----+-------+-----------+
|   G|  false|         25|
|   B|  false|          2|
|   E|   null|         21|
|   B|   null|          2|
|   D|   null|          8|
+----+-------+-----------+
only showing top 5 rows



In [70]:
expr2 = f.mean(f.col('vechime') + 1).cast('decimal(5,2)').alias('mean_vechime_plus_one')
df_customers.groupby('zona', 'inactiv').agg(expr2).show(5)

+----+-------+---------------------+
|zona|inactiv|mean_vechime_plus_one|
+----+-------+---------------------+
|   G|  false|                29.57|
|   B|  false|                 3.00|
|   E|   null|                20.00|
|   B|   null|                 3.33|
|   D|   null|                12.59|
+----+-------+---------------------+
only showing top 5 rows



In [71]:
expr3 = f.collect_list('vechime').alias('all values array')       # returneaza un array din toate valorile
df_customers\
    .groupby('zona', 'inactiv')\
    .agg(expr3)\
    .show(5,truncate=False )

+----+-------+----------------------------------------------------------------------------------------------------------------+
|zona|inactiv|all values array                                                                                                |
+----+-------+----------------------------------------------------------------------------------------------------------------+
|G   |false  |[25, 30, 32, 32, 25, 32, 24]                                                                                    |
|B   |false  |[2]                                                                                                             |
|E   |null   |[21, 21, 18, 23, 15, 21, 17, 20, 22, 18, 15, 15, 15, 23, 21, 17, 15, 17, 23, 15, 18, 22, 21, 15, 21, 22, 18, 23]|
|B   |null   |[2, 2, 2, 2, 1, 3, 2, 1, 3, 2, 1, 1, 3, 1, 1, 1, 3, 11]                                                         |
|D   |null   |[8, 13, 8, 9, 11, 11, 8, 13, 11, 10, 14, 10, 14, 14, 11, 8, 24]                           

In [73]:
expr4 = f.size(\
               f.collect_set(  f.col('vechime') + 1 )\    # array cu valorile unice
               )\
               .alias('vechime uniques values ')
df_customers.groupby('zona', 'inactiv').agg(expr4).show(5)

+----+-------+-----------------------+
|zona|inactiv|vechime uniques values |
+----+-------+-----------------------+
|   G|  false|                      4|
|   B|  false|                      1|
|   E|   null|                      7|
|   B|   null|                      4|
|   D|   null|                      7|
+----+-------+-----------------------+
only showing top 5 rows



In [74]:
# agg

new_data_df = (
    df_customers
    .groupby('zona')
    .agg(
        f.sum('vechime').alias('suma'),
        f.mean('vechime').cast('decimal(5,2)').alias('medie')
        )
).show(5)

+----+----+-----+
|zona|suma|medie|
+----+----+-----+
|   E| 575|19.17|
|   B|  51| 2.55|
|   D| 197|11.59|
|   C|  71| 6.45|
|   A|  39| 2.17|
+----+----+-----+
only showing top 5 rows



In [75]:
# Deduplicare
 # după valorile coloanelor

distinct_df = df_customers.select('nume').distinct()
distinct_df.show(5)

+------+
|  nume|
+------+
|Bogdan|
|Codrin|
| Nadia|
|Florin|
|Lucian|
+------+
only showing top 5 rows



In [76]:
# un nou dataframe cu valorile distincte din setul de date.
dedup_df = df_customers.dropDuplicates(['nume'])
dedup_df.show(5)

+------+------+---------------+-------+-------+----+--------------------+
|  nume|varsta|       ocupatie|vechime|inactiv|zona|               extra|
+------+------+---------------+-------+-------+----+--------------------+
|Bogdan|    50|      Farmacist|     32|  false|   G|[EV, Sauna, 5G Ro...|
|Codrin|    51|      Farmacist|     22|  false|   E|                null|
|Florin|    20|       Educator|      0|   true|   A|                null|
| Nadia|    40|Manager proiect|     15|   null|   E|         [XBOX, WII]|
|Lucian|    49|Designer grafic|     30|  false|   G|[3D Printer, XBOX...|
+------+------+---------------+-------+-------+----+--------------------+
only showing top 5 rows



In [96]:
#

from pyspark.sql import SparkSession, functions as f

# path = '/content/drive/MyDrive/Colab Notebooks/Data/practice/json'
# data_df = spark.read.format('json').load(path)

grouped_data_df = (
    df_customers
    .withColumn('varsta_contractare', f .col('varsta') - f.col('vechime'))
    .groupBy('varsta_contractare')
    .agg(
        f.sum(\
              f.coalesce(f.col('inactiv'), f.lit(False))\
              .cast('integer'))\
        .alias('nr_contracte_inactive'),
        f.mean(f.col('vechime')).alias('vechime_medie')
        )
     )

# path_result = '/content/drive/MyDrive/Colab Notebooks/Data/output/json'
# grouped_data_df.write.format('json').save(path_result)
grouped_data_df.show()

+------------------+---------------------+------------------+
|varsta_contractare|nr_contracte_inactive|     vechime_medie|
+------------------+---------------------+------------------+
|                26|                    0|7.7368421052631575|
|                22|                    1|             14.25|
|                13|                    0|              27.0|
|                20|                    4|               4.8|
|                19|                    3|10.666666666666666|
|                23|                    0|           15.4375|
|                25|                    0| 9.476190476190476|
|                24|                    1|           10.1875|
|                29|                    0|              22.0|
|                21|                    0|              15.4|
|                32|                    0|               6.0|
|                18|                    3|18.142857142857142|
+------------------+---------------------+------------------+



In [78]:
## PRACTICE CURS 3

from pyspark.sql import SparkSession, functions as f



In [ ]:
df_customers = spark.read.format('parquet').load('/content/drive/MyDrive/Colab Notebooks/Data/Course_III/customers_cleaned/practice/parquet')
df_tariff = spark.read.format('parquet').load('/content/drive/MyDrive/Colab Notebooks/Data/Course_III/tariff/practice/parquet')
df_zone_source = spark.read.format('parquet').load('/content/drive/MyDrive/Colab Notebooks/Data/Course_III/zone_source/parquet')
df_zone_type =  spark.read.format('parquet').load('/content/drive/MyDrive/Colab Notebooks/Data/Course_III/zone_type/parquet')

In [ ]:
df_customers.show(5)
df_tariff.show(5)
df_zone_source.show(5)
df_zone_type.show(5)

+--------+------+------------------+-------+-------+----+--------------------+
|    nume|varsta|          ocupatie|vechime|inactiv|zona|               extra|
+--------+------+------------------+-------+-------+----+--------------------+
| Gabriel|    44|Specialist vânzări|     21|   null|   E|[5G Router, Sauna...|
|  George|    46|   Agent imobiliar|     25|  false|   G|[5G Router, 5G Ro...|
|Gheorghe|    34|     Specialist HR|      8|   null|   D|         [PC, Sauna]|
| Grigore|    44|Specialist vânzări|     21|   null|   E|[WII, Piscina, Pi...|
|   Horia|    42|          Arhitect|     18|   null|   E|       [Piscina, AC]|
+--------+------+------------------+-------+-------+----+--------------------+
only showing top 5 rows

+---------+--------+
|     nume|   tarif|
+---------+--------+
|   Andrei| dinamic|
|Alexandru| dinamic|
|   Adrian|   zonal|
|     Alin|standard|
|    Anton|   zonal|
+---------+--------+
only showing top 5 rows

+----+--------+---+
|zona|   sursa|ani|
+----+----

In [ ]:
""""
Gruparea și Agregarea datelor - Dorim să obținem mai multe perspective de ansamblu asupra clienților.

1- Afișarea numărului de oameni, vârsta medie, vechimea medie și procentajul de contracte active per zonă.
- Se recomandă folosirea funcțiilor count și mean. Dacă nu e specificat, atunci contractul e activ.

""""

In [ ]:
df_stats_per_zone = (
    df_customers
    .groupBy('zona').agg(
        f.count('nume').alias('nr de oameni'),
        f.mean('varsta').alias('varsta medie'),
        f.mean('vechime').alias('vachime medie'),
         (
             f.sum(f.coalesce(f.col('inactiv'), f.lit(False)).cast('integer'))  /
             f.count('nume')
             ).alias('procent contr active')
    )
)


In [ ]:
df_stats_per_zone.show()

+----+------------+------------------+------------------+--------------------+
|zona|nr de oameni|      varsta medie|     vachime medie|procent contr active|
+----+------------+------------------+------------------+--------------------+
|   E|          30| 42.93333333333333|19.166666666666668|                 0.0|
|   B|          20|              26.5|              2.55|                0.05|
|   D|          17| 36.94117647058823|11.588235294117647|                 0.0|
|   C|          11| 32.09090909090909| 6.454545454545454|                 0.0|
|   A|          18|              22.0|2.1666666666666665|  0.6111111111111112|
|   G|          14|45.214285714285715|24.357142857142858|                 0.0|
+----+------------+------------------+------------------+--------------------+



In [ ]:
df_stats_per_zone.explain('extended')

== Parsed Logical Plan ==
'Aggregate ['zona], [unresolvedalias('zona, None), count('nume) AS nr de oameni#2949, avg('varsta) AS varsta medie#2951, avg('vechime) AS vachime medie#2953, (sum(cast(coalesce('inactiv, false) as int)) / count('nume)) AS procent contr active#2956]
+- Relation[nume#2779,varsta#2780,ocupatie#2781,vechime#2782,inactiv#2783,zona#2784,extra#2785] parquet

== Analyzed Logical Plan ==
zona: string, nr de oameni: bigint, varsta medie: double, vachime medie: double, procent contr active: double
Aggregate [zona#2784], [zona#2784, count(nume#2779) AS nr de oameni#2949L, avg(cast(varsta#2780 as bigint)) AS varsta medie#2951, avg(cast(vechime#2782 as bigint)) AS vachime medie#2953, (cast(sum(cast(cast(coalesce(inactiv#2783, false) as int) as bigint)) as double) / cast(count(nume#2779) as double)) AS procent contr active#2956]
+- Relation[nume#2779,varsta#2780,ocupatie#2781,vechime#2782,inactiv#2783,zona#2784,extra#2785] parquet

== Optimized Logical Plan ==
Aggregate [zon

In [92]:
"""
2- Afișarea numărului de oameni și vârsta medie per consumator (EV, PV, etc.)
-Se recomandă folosirea funcției explode, count și mean.
"""

df_selection =  df_customers.select('nume','varsta','extra')
df_selection.show(5)

+--------+------+--------------------+
|    nume|varsta|               extra|
+--------+------+--------------------+
| Gabriel|    44|[5G Router, Sauna...|
|  George|    46|[5G Router, 5G Ro...|
|Gheorghe|    34|         [PC, Sauna]|
| Grigore|    44|[WII, Piscina, Pi...|
|   Horia|    42|       [Piscina, AC]|
+--------+------+--------------------+
only showing top 5 rows



In [93]:
df_consumatori = df_selection.withColumn('consumator',f.explode('extra'))
df_consumatori.show(15)

+--------+------+--------------------+----------+
|    nume|varsta|               extra|consumator|
+--------+------+--------------------+----------+
| Gabriel|    44|[5G Router, Sauna...| 5G Router|
| Gabriel|    44|[5G Router, Sauna...|     Sauna|
| Gabriel|    44|[5G Router, Sauna...|        PV|
|  George|    46|[5G Router, 5G Ro...| 5G Router|
|  George|    46|[5G Router, 5G Ro...| 5G Router|
|  George|    46|[5G Router, 5G Ro...|       WII|
|Gheorghe|    34|         [PC, Sauna]|        PC|
|Gheorghe|    34|         [PC, Sauna]|     Sauna|
| Grigore|    44|[WII, Piscina, Pi...|       WII|
| Grigore|    44|[WII, Piscina, Pi...|   Piscina|
| Grigore|    44|[WII, Piscina, Pi...|   Piscina|
|   Horia|    42|       [Piscina, AC]|   Piscina|
|   Horia|    42|       [Piscina, AC]|        AC|
|    Ilie|    26|                [EV]|        EV|
|     Ion|    27|               [PS5]|       PS5|
+--------+------+--------------------+----------+
only showing top 15 rows



In [94]:
df_consumatori_stats = (
    df_consumatori
    .groupBy('consumator')\
    .agg(
        f.countDistinct(f.col('nume')).alias('nr persoane'),\
        f.mean('varsta').cast('decimal(5,2)').alias('varsta medie')\
        ))
df_consumatori_stats.show()

+----------+-----------+------------+
|consumator|nr persoane|varsta medie|
+----------+-----------+------------+
|        PC|          9|       43.11|
|3D Printer|         21|       42.38|
| 5G Router|         17|       41.44|
|     Sauna|         17|       42.39|
|      XBOX|         21|       37.67|
|        EV|         14|       38.14|
|   Piscina|         11|       41.75|
|       WII|         15|       40.25|
|        PV|          8|       38.63|
|       PS5|         13|       40.07|
|        AC|         14|       41.29|
+----------+-----------+------------+



In [ ]:
"""
3- Afișarea numărului de oameni, vârsta medie, vechimea medie și lista zonelor (unice) per număr consumatori.
"""


In [ ]:
df_new_select = df_customers.withColumn('consume cnt',f.when(      f.size(f.col('extra')) > 0,      f.size(f.col('extra'))).otherwise(f.lit(0))  )
df_new_select.show(5)

+--------+------+------------------+-------+-------+----+--------------------+-----------+
|    nume|varsta|          ocupatie|vechime|inactiv|zona|               extra|consume cnt|
+--------+------+------------------+-------+-------+----+--------------------+-----------+
| Gabriel|    44|Specialist vânzări|     21|   null|   E|[5G Router, Sauna...|          3|
|  George|    46|   Agent imobiliar|     25|  false|   G|[5G Router, 5G Ro...|          3|
|Gheorghe|    34|     Specialist HR|      8|   null|   D|         [PC, Sauna]|          2|
| Grigore|    44|Specialist vânzări|     21|   null|   E|[WII, Piscina, Pi...|          3|
|   Horia|    42|          Arhitect|     18|   null|   E|       [Piscina, AC]|          2|
+--------+------+------------------+-------+-------+----+--------------------+-----------+
only showing top 5 rows



In [ ]:
df_new = df_new_select.groupBy('consume cnt').agg(
    f.countDistinct('nume').alias('nr de persoane'),
    f.mean('varsta').alias('varsta medie'),
    f.mean('vechime').alias('vechime media'),
    f.collect_list('zona').alias('lista zone')
)
df_new.show()

+-----------+--------------+------------------+------------------+--------------------+
|consume cnt|nr de persoane|      varsta medie|     vechime media|          lista zone|
+-----------+--------------+------------------+------------------+--------------------+
|          1|            25|             30.04|               5.8|[B, B, B, C, B, G...|
|          3|            28|             45.75|24.178571428571427|[G, E, G, G, E, E...|
|          2|            31|38.096774193548384|13.161290322580646|[E, E, E, E, E, D...|
|          0|            26|23.653846153846153|1.6923076923076923|[A, E, B, A, B, A...|
+-----------+--------------+------------------+------------------+--------------------+



In [ ]:
"""
4- Afișarea numărului de oameni, vechimea medie, și numărul mediu de consumatori per categorie de vârstă (15- 19, 20-24, 25-29, ...)
"""
exp =  f.when((f.col('varsta') >= f.lit(15) )&( f.col('varsta') <= f.lit(39)) , f.lit('15 - 39 ani')).when((f.col('varsta') >= f.lit(40) )&( f.col('varsta') <= f.lit(59) ), f.lit('40 - 59 ani' )).otherwise(f.lit(' >= 60'))

df_add_varstagroup = (
    df_customers
    .withColumn('categ varsta', exp)
)

df_add_varstagroup.show(15)

+---------+------+--------------------+-------+-------+----+--------------------+------------+
|     nume|varsta|            ocupatie|vechime|inactiv|zona|               extra|categ varsta|
+---------+------+--------------------+-------+-------+----+--------------------+------------+
|  Gabriel|    44|  Specialist vânzări|     21|   null|   E|[5G Router, Sauna...| 40 - 59 ani|
|   George|    46|     Agent imobiliar|     25|  false|   G|[5G Router, 5G Ro...| 40 - 59 ani|
| Gheorghe|    34|       Specialist HR|      8|   null|   D|         [PC, Sauna]| 15 - 39 ani|
|  Grigore|    44|  Specialist vânzări|     21|   null|   E|[WII, Piscina, Pi...| 40 - 59 ani|
|    Horia|    42|            Arhitect|     18|   null|   E|       [Piscina, AC]| 40 - 59 ani|
|     Ilie|    26|     Vânzător retail|      2|   null|   B|                [EV]| 15 - 39 ani|
|      Ion|    27|     Vânzător retail|      2|   null|   B|               [PS5]| 15 - 39 ani|
|    Ionel|    45|       Inginer civil|     23|   

In [ ]:
df_calc_by_varsta =  df_add_varstagroup.groupBy('categ varsta').agg(f.count('nume'))

df_calc_by_varsta.show()

+------------+-----------+
|categ varsta|count(nume)|
+------------+-----------+
| 15 - 39 ani|         65|
| 40 - 59 ani|         45|
+------------+-----------+



In [ ]:
"""
Gruparea și Transformarea datelor
Dorim să obținem și mai multe perspective de ansamblu asupra clienților.
1 -  Adăugarea coloanei „numar_ani_ramasi” egală cu diferența dintre vechimea persoanei și vechimea medie a
persoanelor din zona din care face parte.

"""

In [106]:
df_medie_vechime_pe_zona = (
    df_customers\
      .groupBy('zona')\
      .agg(\
          f.mean('vechime')\
          .cast('decimal(10,2)')\
          .alias('vechime_medie')\
          )
)
df_medie_vechime_pe_zona .show()

+----+-------------+
|zona|vechime_medie|
+----+-------------+
|   E|        19.17|
|   B|         2.55|
|   D|        11.59|
|   C|         6.45|
|   A|         2.17|
|   G|        24.36|
+----+-------------+



In [109]:
df_customers_ani_ramasi = (
df_customers\
      .join(df_medie_vechime_pe_zona,on = 'zona')\
      .withColumn('numar_ani_ramasi',f.col('vechime') - f.col('vechime_medie'))\
)
df_customers_ani_ramasi.show()

+----+---------+------+--------------------+-------+-------+--------------------+-------------+----------------+
|zona|     nume|varsta|            ocupatie|vechime|inactiv|               extra|vechime_medie|numar_ani_ramasi|
+----+---------+------+--------------------+-------+-------+--------------------+-------------+----------------+
|   E|  Gabriel|    44|  Specialist vânzări|     21|   null|[5G Router, Sauna...|        19.17|            1.83|
|   G|   George|    46|     Agent imobiliar|     25|  false|[5G Router, 5G Ro...|        24.36|            0.64|
|   D| Gheorghe|    34|       Specialist HR|      8|   null|         [PC, Sauna]|        11.59|           -3.59|
|   E|  Grigore|    44|  Specialist vânzări|     21|   null|[WII, Piscina, Pi...|        19.17|            1.83|
|   E|    Horia|    42|            Arhitect|     18|   null|       [Piscina, AC]|        19.17|           -1.17|
|   B|     Ilie|    26|     Vânzător retail|      2|   null|                [EV]|         2.55| 

In [ ]:
"""
2 -  Adăugarea coloanei „peste_medie” booleană cu True dacă coloana „numar_ani_ramasi” este pozitivă și False
dacă nu.

"""

In [111]:
df_add_flag_peste_medie = (
    df_customers_ani_ramasi\
    .withColumn('peste_medie',\
                f.when(\
                       f.col('numar_ani_ramasi') > 0,\
                       f.lit(True))\
                .otherwise(f.lit(False))\
                )
)
df_add_flag_peste_medie.show()

+----+---------+------+--------------------+-------+-------+--------------------+-------------+----------------+-----------+
|zona|     nume|varsta|            ocupatie|vechime|inactiv|               extra|vechime_medie|numar_ani_ramasi|peste_medie|
+----+---------+------+--------------------+-------+-------+--------------------+-------------+----------------+-----------+
|   E|  Gabriel|    44|  Specialist vânzări|     21|   null|[5G Router, Sauna...|        19.17|            1.83|       true|
|   G|   George|    46|     Agent imobiliar|     25|  false|[5G Router, 5G Ro...|        24.36|            0.64|       true|
|   D| Gheorghe|    34|       Specialist HR|      8|   null|         [PC, Sauna]|        11.59|           -3.59|      false|
|   E|  Grigore|    44|  Specialist vânzări|     21|   null|[WII, Piscina, Pi...|        19.17|            1.83|       true|
|   E|    Horia|    42|            Arhitect|     18|   null|       [Piscina, AC]|        19.17|           -1.17|      false|


In [ ]:
"""

3 -  Grupați după coloana „peste_medie” și afișați numărul total de persoane din fiecare categorie.


"""

In [115]:
df_add_flag_peste_medie\
      .groupBy('peste_medie')\
      .agg(f.count('nume').alias('numar de persoane'))\
      .show()

+-----------+-----------------+
|peste_medie|numar de persoane|
+-----------+-----------------+
|       true|               42|
|      false|               68|
+-----------+-----------------+



In [ ]:
# Descrierea obiectivului exemplului de cod din curs de la finalul acestei secțiuni.

In [84]:
from pyspark.sql import SparkSession, functions as f, Window

# path = '/content/drive/MyDrive/Colab Notebooks/Data/practice/json'
# data_df = spark.read.format('json').load(path)

window = Window.partitionBy('unitate_varsta').orderBy('vechime').rangeBetween(-2, 2)

processed_data_df = (
    df_customers
    .withColumn('inactiv', f.coalesce(f.col('inactiv'), f.lit(False)))
    .withColumn('unitate_varsta', f.floor(f.col('varsta') / 5))
    .withColumn('nr_contracte_similare_inactive', f.sum(f.col('inactiv').cast('integer')).over(window))
    .withColumn('nr_contracte_similare', f.count(f.col('inactiv')).over(window))
    .withColumn('probabilitate_inactivare',\
                (f .col('nr_contracte_similare_inactive') / f.col('nr_contracte_similare')).cast('decimal(5,4)')\
    ))\
    .show(50)#.drop('extra','nume','ocupatie').filter(f.col('unitate_varsta') == f.lit(4))



+---------+------+--------------------+-------+-------+----+--------------------+--------------+------------------------------+---------------------+------------------------+
|     nume|varsta|            ocupatie|vechime|inactiv|zona|               extra|unitate_varsta|nr_contracte_similare_inactive|nr_contracte_similare|probabilitate_inactivare|
+---------+------+--------------------+-------+-------+----+--------------------+--------------+------------------------------+---------------------+------------------------+
| Victoria|    38|        Medic primar|      6|  false|   G|                null|             7|                             0|                    1|                  0.0000|
|      Ana|    35|       Specialist HR|      9|  false|   D|    [EV, 3D Printer]|             7|                             0|                    7|                  0.0000|
|    Laura|    36|  Muncitor alimentar|     10|  false|   D|[3D Printer, 3D P...|             7|                             

In [ ]:
"""
Uniunea și asocierea datelor
Folosind informațiile suplimentare despre clienți și zona din care fac parte, calculați următoarele statistici:
1 -  Afișarea numărului de oameni, vârsta medie, vechimea medie și procentajul de contracte active per tarif.

"""

In [116]:
df_customers.show(5)
df_tariff.show(5)

+--------+------+------------------+-------+-------+----+--------------------+
|    nume|varsta|          ocupatie|vechime|inactiv|zona|               extra|
+--------+------+------------------+-------+-------+----+--------------------+
| Gabriel|    44|Specialist vânzări|     21|   null|   E|[5G Router, Sauna...|
|  George|    46|   Agent imobiliar|     25|  false|   G|[5G Router, 5G Ro...|
|Gheorghe|    34|     Specialist HR|      8|   null|   D|         [PC, Sauna]|
| Grigore|    44|Specialist vânzări|     21|   null|   E|[WII, Piscina, Pi...|
|   Horia|    42|          Arhitect|     18|   null|   E|       [Piscina, AC]|
+--------+------+------------------+-------+-------+----+--------------------+
only showing top 5 rows

+---------+--------+
|     nume|   tarif|
+---------+--------+
|   Andrei| dinamic|
|Alexandru| dinamic|
|   Adrian|   zonal|
|     Alin|standard|
|    Anton|   zonal|
+---------+--------+
only showing top 5 rows



In [124]:
# procentajul de contracte active per tarif.
df_customers.join(df_tariff,on = 'nume')\
.groupBy('tarif')\
.agg(\
     f.count('nume').alias('nr de persoane'),\
     f.mean('varsta').cast('decimal(7,2)').alias('varsta medie'),\
     f.mean('vechime').cast('decimal(7,2)').alias('vechime medie'),\
     (f.sum(f.coalesce(f.col('inactiv'), f.lit(False)).cast('integer')) / f.count('nume'))\
     .cast('decimal(7,2)')\
     .alias('procent contr active')\
).show()

+--------+--------------+------------+-------------+--------------------+
|   tarif|nr de persoane|varsta medie|vechime medie|procent contr active|
+--------+--------------+------------+-------------+--------------------+
| dinamic|            30|       34.17|         8.83|                0.03|
|   zonal|            44|       43.84|        21.52|                0.00|
|standard|            38|       25.11|         2.45|                0.29|
+--------+--------------+------------+-------------+--------------------+



In [ ]:
"""

2 -  Afișarea numărului de oameni, vârsta medie, vechimea medie și procentajul de contracte active per mediu.

"""

In [127]:
df_customers.show(5)
df_zone_type.show(5)

+--------+------+------------------+-------+-------+----+--------------------+
|    nume|varsta|          ocupatie|vechime|inactiv|zona|               extra|
+--------+------+------------------+-------+-------+----+--------------------+
| Gabriel|    44|Specialist vânzări|     21|   null|   E|[5G Router, Sauna...|
|  George|    46|   Agent imobiliar|     25|  false|   G|[5G Router, 5G Ro...|
|Gheorghe|    34|     Specialist HR|      8|   null|   D|         [PC, Sauna]|
| Grigore|    44|Specialist vânzări|     21|   null|   E|[WII, Piscina, Pi...|
|   Horia|    42|          Arhitect|     18|   null|   E|       [Piscina, AC]|
+--------+------+------------------+-------+-------+----+--------------------+
only showing top 5 rows

+----+--------+
|zona|   mediu|
+----+--------+
|   D|Suburban|
|   B|Suburban|
|   A|   Urban|
|   C|   Urban|
|   E|   Rural|
+----+--------+



In [128]:
df_customers.join(df_zone_type,on = 'zona')\
.groupBy('mediu')\
.agg(\
     f.count('nume').alias('nr de persoane'),\
     f.mean('varsta').cast('decimal(7,2)').alias('varsta medie'),\
     f.mean('vechime').cast('decimal(7,2)').alias('vechime medie'),\
     (f.sum(f.coalesce(f.col('inactiv'), f.lit(False)).cast('integer')) / f.count('nume'))\
     .cast('decimal(7,2)')\
     .alias('procent contr active')\
).show()

+--------+--------------+------------+-------------+--------------------+
|   mediu|nr de persoane|varsta medie|vechime medie|procent contr active|
+--------+--------------+------------+-------------+--------------------+
|   Urban|            29|       25.83|         3.79|                0.38|
|Suburban|            37|       31.30|         6.70|                0.03|
|   Rural|            30|       42.93|        19.17|                0.00|
+--------+--------------+------------+-------------+--------------------+



In [ ]:
"""

3 -  Afișarea numărului de oameni, vârsta medie, vechimea medie și procentajul de contracte active per sursă de
energie. Dacă această informație nu este valabilă pentru o persoană, este considerată „fosila”.

"""

In [129]:
df_customers.show(5)
df_zone_source.show(5)

+--------+------+------------------+-------+-------+----+--------------------+
|    nume|varsta|          ocupatie|vechime|inactiv|zona|               extra|
+--------+------+------------------+-------+-------+----+--------------------+
| Gabriel|    44|Specialist vânzări|     21|   null|   E|[5G Router, Sauna...|
|  George|    46|   Agent imobiliar|     25|  false|   G|[5G Router, 5G Ro...|
|Gheorghe|    34|     Specialist HR|      8|   null|   D|         [PC, Sauna]|
| Grigore|    44|Specialist vânzări|     21|   null|   E|[WII, Piscina, Pi...|
|   Horia|    42|          Arhitect|     18|   null|   E|       [Piscina, AC]|
+--------+------+------------------+-------+-------+----+--------------------+
only showing top 5 rows

+----+--------+---+
|zona|   sursa|ani|
+----+--------+---+
|   D|nucleara| 20|
|   B|nucleara| 16|
|   C| eoliana|  3|
|   D|  fosila| 45|
|   C|  fosila| 45|
+----+--------+---+
only showing top 5 rows



In [132]:
df_customers\
.join(df_zone_source,on = 'zona',how='left')\
.withColumn('sursa',f.coalesce(f.col('sursa'),f.lit('fosila')))\
.groupBy('sursa')\
.agg(\
     f.count('nume').alias('nr de persoane'),\
     f.mean('varsta').cast('decimal(7,2)').alias('varsta medie'),\
     f.mean('vechime').cast('decimal(7,2)').alias('vechime medie'),\
     (f.sum(f.coalesce(f.col('inactiv'), f.lit(False)).cast('integer')) / f.count('nume'))\
     .cast('decimal(7,2)')\
     .alias('procent contr active')\
).show()

+--------+--------------+------------+-------------+--------------------+
|   sursa|nr de persoane|varsta medie|vechime medie|procent contr active|
+--------+--------------+------------+-------------+--------------------+
|nucleara|            37|       31.30|         6.70|                0.03|
|  fosila|            72|       40.31|        16.44|                0.00|
| eoliana|            11|       32.09|         6.45|                0.00|
|  solara|            37|       31.30|         6.70|                0.03|
|   hidro|            18|       22.00|         2.17|                0.61|
+--------+--------------+------------+-------------+--------------------+



In [ ]:
"""
4 -  Afișarea numărului de oameni per sursa de energie și tarif, sortate descrescător după număr doar pentru
persoanele cu o vechime mai mică decât ani de utilizare a sursei de energie.
 Va fi nevoie de efectuat două join-uri aici, unul după altul.
 """

In [151]:
df_customers\
.join(df_tariff, on =   'nume'  )\
.join(df_zone_source,on = 'zona',how='left')\
.withColumn('sursa',f.coalesce(f.col('sursa'),f.lit('fosila')))\
.withColumn('ani',f.coalesce(f.col('ani'),f.lit(0)))\
.where( f.col('vechime') < f.col('ani') )\
.groupBy('sursa','tarif')\
.agg(\
     f.count('nume').alias('nr de persoane'))\
.show()


+--------+--------+--------------+
|   sursa|   tarif|nr de persoane|
+--------+--------+--------------+
|nucleara|   zonal|             1|
|  fosila|standard|             1|
|  solara|standard|             7|
|   hidro| dinamic|             1|
|nucleara| dinamic|            17|
|  fosila| dinamic|            26|
|   hidro|standard|            16|
|  fosila|   zonal|             2|
|nucleara|standard|            19|
+--------+--------+--------------+

